In [1]:
!pip install rasterio geopandas gdal matplotlib
!pip install exifread
!pip install georinex

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 32.3 MB/s eta 0:00:00


### import packages

In [2]:
import os
from osgeo import gdal
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rasterio

#### Function of RAT --> Multispectral

In [3]:
# Function to get multispectral groups
def get_multispectral_groups(input_folder):
    files = sorted(os.listdir(input_folder))  # Sort files to ensure order
    groups = []
    current_group = []

    for file in files:
        if file.endswith(".JPG"):  # Start of a new group
            if current_group:  # Save the current group
                groups.append(current_group)
            current_group = [os.path.join(input_folder, file)]  # Start a new group
        elif file.endswith(".TIF"):  # Add TIF files to the current group
            current_group.append(os.path.join(input_folder, file))

    if current_group:  # Add the last group
        groups.append(current_group)

    # Validate groups (each group must have 6 files: 1 JPG + 5 TIFs)
    valid_groups = [group for group in groups if len(group) == 6]
    return valid_groups

In [8]:
def raw_to_mulspec(input_folder, output_folder, lbl_name):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get all multispectral groups
    groups = get_multispectral_groups(input_folder)

    print(f"Found {len(groups)} multispectral groups.")

    # Process each group to create multispectral images
    for group_idx, group in enumerate(groups, start=1):
        jpg_file = group[0]  # The JPG file (not used in processing)
        tif_files = group[1:]  # The 5 TIF files for Blue, Green, Red, RedEdge, NIR

        # Define band names and paths
        bands = {
            "Blue": tif_files[0],
            "Green": tif_files[1],
            "Red": tif_files[2],
            "RedEdge": tif_files[3],
            "NIR": tif_files[4]
        }

        # Open each band as a GDAL dataset
        datasets = {name: gdal.Open(path) for name, path in bands.items()}

        # Check that all bands have the same dimensions
        width = datasets["Blue"].RasterXSize
        height = datasets["Blue"].RasterYSize
        for name, dataset in datasets.items():
            if dataset.RasterXSize != width or dataset.RasterYSize != height:
                raise ValueError(f"Band {name} in group {group_idx} does not have the same dimensions!")

        # Create output file
        output_file = os.path.join(output_folder, f"{lbl_name}_{group_idx}.tif")
        driver = gdal.GetDriverByName("GTiff")
        output = driver.Create(output_file, width, height, len(bands), gdal.GDT_Float32)

        # Write each band to the output file with proper labeling
        for i, (name, dataset) in enumerate(datasets.items(), start=1):
            print(f"Group {group_idx}: Writing band {name} to output file as layer {i}...")
            band_data = dataset.GetRasterBand(1).ReadAsArray()
            output.GetRasterBand(i).WriteArray(band_data)
            output.GetRasterBand(i).SetDescription(name)  # Set band label (metadata)

        # Save and close the output file
        output.FlushCache()
        del output

        print(f"Group {group_idx}: Multispectral image saved to: {output_file}")

    print("All multispectral images have been processed.")

In [20]:
from pathlib import Path

base_path_1 = Path('/content/drive/MyDrive/Research Methodology/[1] Galoya Plantation Research/Data Collection/Data/Data/Galoya Data/')
base_path_2 = Path('/content/drive/MyDrive/Research Methodology/[1] Galoya Plantation Research/Data Collection/Data/')

germination_input_path_1 = os.path.join(base_path_2,"108FPLAN")
germination_input_path_2 = os.path.join(base_path_2,"109FPLAN")
germination_input_path_3 = os.path.join(base_path_2,"110FPLAN")


tillering_input_path_1 = os.path.join(base_path_1,"115FPLAN")
tillering_input_path_2 = os.path.join(base_path_1,"116FPLAN")

grand_growth_input_path_1 = os.path.join(base_path_1,"117FPLAN")
grand_growth_input_path_2 = os.path.join(base_path_1,"118FPLAN")

### germination

In [22]:
output_path = Path('/content/drive/MyDrive/Colab Notebooks/Research/TEST_2/MULTISPECTRAL/germination')
raw_to_mulspec(germination_input_path_1, output_path, 'germination_110FPLAN')
raw_to_mulspec(germination_input_path_2, output_path, 'germination_111FPLAN')
raw_to_mulspec(germination_input_path_3, output_path, 'germination_112FPLAN')

Found 45 multispectral groups.
Group 1: Writing band Blue to output file as layer 1...
Group 1: Writing band Green to output file as layer 2...
Group 1: Writing band Red to output file as layer 3...
Group 1: Writing band RedEdge to output file as layer 4...
Group 1: Writing band NIR to output file as layer 5...
Group 1: Multispectral image saved to: /content/drive/MyDrive/Colab Notebooks/Research/TEST_2/MULTISPECTRAL/germination/germination_110FPLAN_1.tif
Group 2: Writing band Blue to output file as layer 1...
Group 2: Writing band Green to output file as layer 2...
Group 2: Writing band Red to output file as layer 3...
Group 2: Writing band RedEdge to output file as layer 4...
Group 2: Writing band NIR to output file as layer 5...
Group 2: Multispectral image saved to: /content/drive/MyDrive/Colab Notebooks/Research/TEST_2/MULTISPECTRAL/germination/germination_110FPLAN_2.tif
Group 3: Writing band Blue to output file as layer 1...
Group 3: Writing band Green to output file as layer 2..

### tillering

In [9]:
output_path = Path('/content/drive/MyDrive/Colab Notebooks/Research/TEST_2/MULTISPECTRAL/tillering')
raw_to_mulspec(tillering_input_path_1, output_path, 'tillering_115FPLAN')
raw_to_mulspec(tillering_input_path_2, output_path, 'tillering_116FPLAN')

Found 99 multispectral groups.
Group 1: Writing band Blue to output file as layer 1...
Group 1: Writing band Green to output file as layer 2...
Group 1: Writing band Red to output file as layer 3...
Group 1: Writing band RedEdge to output file as layer 4...
Group 1: Writing band NIR to output file as layer 5...
Group 1: Multispectral image saved to: /content/drive/MyDrive/Colab Notebooks/Research/TEST_2/MULTISPECTRAL/tillering/tillering_115FPLAN_1.tif
Group 2: Writing band Blue to output file as layer 1...
Group 2: Writing band Green to output file as layer 2...
Group 2: Writing band Red to output file as layer 3...
Group 2: Writing band RedEdge to output file as layer 4...
Group 2: Writing band NIR to output file as layer 5...
Group 2: Multispectral image saved to: /content/drive/MyDrive/Colab Notebooks/Research/TEST_2/MULTISPECTRAL/tillering/tillering_115FPLAN_2.tif
Group 3: Writing band Blue to output file as layer 1...
Group 3: Writing band Green to output file as layer 2...
Group 

### grand_growth

In [13]:
output_path = Path('/content/drive/MyDrive/Colab Notebooks/Research/TEST_2/MULTISPECTRAL/grand_growth')
raw_to_mulspec(grand_growth_input_path_1, output_path, 'grand_growth_117FPLAN')
raw_to_mulspec(grand_growth_input_path_2, output_path, 'grand_growth_118FPLAN')

Found 97 multispectral groups.
Group 1: Writing band Blue to output file as layer 1...
Group 1: Writing band Green to output file as layer 2...
Group 1: Writing band Red to output file as layer 3...
Group 1: Writing band RedEdge to output file as layer 4...
Group 1: Writing band NIR to output file as layer 5...
Group 1: Multispectral image saved to: /content/drive/MyDrive/Colab Notebooks/Research/TEST_2/MULTISPECTRAL/grand_growth/grand_growth_117FPLAN_1.tif
Group 2: Writing band Blue to output file as layer 1...
Group 2: Writing band Green to output file as layer 2...
Group 2: Writing band Red to output file as layer 3...
Group 2: Writing band RedEdge to output file as layer 4...
Group 2: Writing band NIR to output file as layer 5...
Group 2: Multispectral image saved to: /content/drive/MyDrive/Colab Notebooks/Research/TEST_2/MULTISPECTRAL/grand_growth/grand_growth_117FPLAN_2.tif
Group 3: Writing band Blue to output file as layer 1...
Group 3: Writing band Green to output file as layer